In [1]:
import transformers
import pandas
import numpy
import torch

/home/jerko/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pandas.read_parquet('raw_data/RACE/train-00000-of-00001.parquet')

In [3]:
dataset

,example_id,article,answer,question,options
0,high19088.txt,Last week I talked with some of my students ab...,C,We can know from the passage that the author w...,"[doctor, model, teacher, reporter]"
1,high19088.txt,Last week I talked with some of my students ab...,C,Many graduates today turn to cosmetic surgery ...,"[marry a better man/woman, become a model, get..."
2,high19088.txt,Last week I talked with some of my students ab...,D,"According to the passage, the author believes ...","[everyone should purchase perfection, whatever..."
3,high19088.txt,Last week I talked with some of my students ab...,B,Which' s the best title for the passage?.,"[Young Graduates Have Higher Expectations, You..."
4,high15596.txt,"YUZHOU, HENAN -An accident in a central China ...",B,What could be the best title for this passage?,"[Death Toll Rises in an Accident in China, A C..."
...,...,...,...,...,...
87861,middle2313.txt,"Last year when Tom graduated from school, he c...",B,Tom came to the station to _ .,"[find a job, take the train home, sell the tic..."
87862,middle2313.txt,"Last year when Tom graduated from school, he c...",C,The woman walked to Tom because _ .,"[she was Tom's old friend, her child wanted to..."
87863,middle2313.txt,"Last year when Tom graduated from school, he c...",D,Tom didn't take the train because _ .,"[he didn't take a train ticket, he missed the ..."
87864,middle2313.txt,"Last year when Tom graduated from school, he c...",B,The old man wanted to _ .,"[lend Tom some money, give Tom a job, say ""Tha..."
